In [1]:
import pandas as pd

# Cargar el dataset desde el archivo Excel
df = pd.read_excel('data_downsampled.xlsx')

# Mostrar las primeras filas y obtener un resumen general del dataset
print(df.head())
print(df.info())
print("Número de registros:", len(df))


   raiz                                            Comment  \
0   NaN  la conferencia para que sepa un poco mas sobre...   
1   NaN  que el gobierno algo busca, están queriendo va...   
2  13.0  Sres. digan también que las causas de este pap...   
3  43.0  En Colombia se murieron 365 niñas en Carmen de...   
4   NaN  exacto eso leí una vez que da autismo cuando t...   

                                        Topic  Topic_c  
0                         Rechazo a la vacuna        0  
1  En contra de la OMS u otras organizaciones        0  
2                         Rechazo a la vacuna        0  
3                         Rechazo a la vacuna        0  
4                         Rechazo a la vacuna        0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   raiz     313 non-null    float64
 1   Comment  608 non-null    object 
 2   Topic    608 non-null   

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Descargar recursos necesarios (solo si no se han descargado)
nltk.download('stopwords')
nltk.download('wordnet')

# Inicializar stopwords y lematizador
stop_words = set(stopwords.words('spanish'))
lemmatizer = WordNetLemmatizer()

def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar caracteres especiales y dígitos
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)
    # Tokenizar
    palabras = texto.split()
    # Eliminar stopwords y aplicar lematización
    palabras_limpias = [lemmatizer.lemmatize(p) for p in palabras if p not in stop_words]
    return ' '.join(palabras_limpias)

# Aplicar la función de limpieza a la columna de comentarios
df['Comment_clean'] = df['Comment'].astype(str).apply(limpiar_texto)

# Visualizar algunos ejemplos de comentarios limpios
print(df[['Comment', 'Comment_clean']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ulewi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ulewi\AppData\Roaming\nltk_data...


                                             Comment  \
0  la conferencia para que sepa un poco mas sobre...   
1  que el gobierno algo busca, están queriendo va...   
2  Sres. digan también que las causas de este pap...   
3  En Colombia se murieron 365 niñas en Carmen de...   
4  exacto eso leí una vez que da autismo cuando t...   

                                       Comment_clean  
0          conferencia sepa ma vacunas consecuencias  
1  gobierno busca queriendo vacunar gente diferen...  
2  sres digan causa papiloma humano cáncer cuello...  
3  colombia murieron niñas carmen bolívar persona...  
4  exacto leí vez da autismo vacunan embarazada p...  


In [9]:
from sentence_transformers import SentenceTransformer

# Cargar un modelo pre-entrenado de Sentence Transformers
modelo_embedding = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para los comentarios limpios
df['embedding_st'] = df['Comment_clean'].apply(lambda x: modelo_embedding.encode(x))

# Mostrar el vector de embedding del primer comentario (resumido)
print("Embedding (Sentence Transformers) del primer comentario:", df['embedding_st'].iloc[0][:5])


c:\Users\ulewi\.conda\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ulewi\.conda\envs\tesis\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ulewi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to ac

Embedding (Sentence Transformers) del primer comentario: [ 0.03728831 -0.00187563 -0.04465014  0.02665996 -0.07623093]


In [ ]:
from openai import OpenAI
import pandas as pd

# Configuramos la clave de API y creamos el cliente
client = OpenAI(api_key="API")

def obtener_embedding_openai(texto):
    respuesta = client.embeddings.create(
        input=texto,
        model="text-embedding-ada-002"
    )
    
    embedding = respuesta.data[0].embedding
    return embedding

# Aplicamos la función creada
df['embedding_openai'] = df['Comment_clean'].apply(obtener_embedding_openai)

# Ejemplo: Mostrar los primeros 5 valores del embedding para el primer comentario
print("Embedding (OpenAI) del primer comentario:", df['embedding_openai'].iloc[0][:5])


Embedding (OpenAI) del primer comentario: [-0.018249183893203735, 0.01009572297334671, 0.0234959926456213, -0.02457502670586109, -0.010702680796384811]


In [14]:
df.to_excel("data_with_embedding.xlsx", index=False)

# Modelamiento

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Extraer la dimensión del embedding (se asume que todos tienen la misma dimensión)
embedding_dim = len(df['embedding_openai'].iloc[0])

# Convertir cada componente del embedding en una columna independiente
for i in range(embedding_dim):
    df[f'emboai_{i}'] = df['embedding_openai'].apply(lambda x: x[i])

# Definir las características y la variable objetivo
features = [f'emboai_{i}' for i in range(embedding_dim)]
target = 'Topic_c'

# Separar el dataset en entrenamiento (80%) y prueba (20%)
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar LazyClassifier para evaluar múltiples modelos rápidamente
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Entrenar y evaluar los modelos
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Mostrar el leaderboard con las métricas (accuracy, tiempo de entrenamiento, etc.)
print("Leaderboard de modelos evaluados:")
print(models)


 97%|█████████▋| 31/32 [00:38<00:02,  2.77s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 249339
[LightGBM] [Info] Number of data points in the train set: 486, number of used features: 1536
[LightGBM] [Info] Start training from score -1.415524
[LightGBM] [Info] Start training from score -1.458821
[LightGBM] [Info] Start training from score -1.334178
[LightGBM] [Info] Start training from score -1.342022
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

100%|██████████| 32/32 [00:47<00:00,  1.49s/it]

Leaderboard de modelos evaluados:
                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
NuSVC                              0.62               0.62    None      0.62   
SVC                                0.61               0.60    None      0.61   
LGBMClassifier                     0.57               0.59    None      0.57   
RandomForestClassifier             0.57               0.57    None      0.56   
PassiveAggressiveClassifier        0.56               0.56    None      0.56   
CalibratedClassifierCV             0.55               0.56    None      0.55   
XGBClassifier                      0.54               0.55    None      0.53   
LogisticRegression                 0.55               0.54    None      0.55   
LinearSVC                          0.52               0.52    None      0.51   
Perceptron                         0.52               0.51    None      0.52   
ExtraT

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier


# Extraer la dimensión del embedding (se asume que todos tienen la misma dimensión)
embedding_dim2 = len(df['embedding_st'].iloc[0])

# Convertir cada componente del embedding en una columna independiente
for i in range(embedding_dim2):
    df[f'embst_{i}'] = df['embedding_st'].apply(lambda x: x[i])

# Definir las características y la variable objetivo
features = [f'embst_{i}' for i in range(embedding_dim2)]
target = 'Topic_c'

# Separar el dataset en entrenamiento (80%) y prueba (20%)
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar LazyClassifier para evaluar múltiples modelos rápidamente
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Entrenar y evaluar los modelos
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Mostrar el leaderboard con las métricas (accuracy, tiempo de entrenamiento, etc.)
print("Leaderboard de modelos evaluados:")
print(models)


 97%|█████████▋| 31/32 [00:20<00:00,  1.94it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003392 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 62347
[LightGBM] [Info] Number of data points in the train set: 486, number of used features: 384
[LightGBM] [Info] Start training from score -1.415524
[LightGBM] [Info] Start training from score -1.458821
[LightGBM] [Info] Start training from score -1.334178
[LightGBM] [Info] Start training from score -1.342022
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

100%|██████████| 32/32 [00:21<00:00,  1.48it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Leaderboard de modelos ev

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
import joblib

# --- Preparación del dataset ---
embedding_dim_st = len(df['embedding_st'].iloc[0])
for i in range(embedding_dim_st):
    df[f'st_emb_{i}'] = df['embedding_st'].apply(lambda x: x[i])

# Definir las características y la variable objetivo para los embeddings de Sentence Transformers
features_st = [f'st_emb_{i}' for i in range(embedding_dim_st)]
target = 'Topic_c'  # Asegúrate de que esta columna esté presente en tu DataFrame

# Dividir el dataset en entrenamiento (80%) y prueba (20%)
X = df[features_st]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Entrenamiento del modelo SGDClassifier ---
modelo_sgd = SGDClassifier(random_state=42)
modelo_sgd.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = modelo_sgd.predict(X_test)

# --- Cálculo de las métricas para cada categoría (one-vs-all) ---
# Convertir y_test a array (si aún no lo es) y obtener el listado de clases
y_test_array = np.array(y_test)
clases = np.unique(np.concatenate((y_test_array, y_pred), axis=0))

# Calcular la matriz de confusión
cm = confusion_matrix(y_test_array, y_pred, labels=clases)
print("Matriz de confusión:")
print(cm)

# Crear un DataFrame para almacenar las métricas
metricas = pd.DataFrame(index=clases, columns=['Sensibilidad', 'Especificidad', 'F1', 'Accuracy'])

# Para cada clase (tratada como la clase positiva frente a todas las demás)
for idx, clase in enumerate(clases):
    TP = cm[idx, idx]                   # Verdaderos positivos para la clase
    FN = cm[idx, :].sum() - TP            # Falsos negativos
    FP = cm[:, idx].sum() - TP            # Falsos positivos
    TN = cm.sum() - (TP + FP + FN)          # Verdaderos negativos

    sensibilidad = TP / (TP + FN) if (TP + FN) > 0 else 0
    especificidad = TN / (TN + FP) if (TN + FP) > 0 else 0
    f1 = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
    accuracy = (TP + TN) / cm.sum()

    metricas.loc[clase] = [sensibilidad, especificidad, f1, accuracy]

print("\nMétricas por cada categoría de 'Topic_c':")
print(metricas)

# --- Guardar el modelo para su uso en Streamlit ---
joblib.dump(modelo_sgd, 'modelo_sgd.pkl')
print("\nEl modelo SGDClassifier se ha guardado en 'modelo_sgd.pkl'")

Matriz de confusión:
[[  8   4   0  24]
 [  0   4   2  28]
 [  0   2   5  21]
 [  2  10   6 198]]

Métricas por cada categoría de 'Topic_c':
  Sensibilidad Especificidad   F1 Accuracy
0         0.22          0.99 0.35     0.90
1         0.12          0.94 0.15     0.85
2         0.18          0.97 0.24     0.90
3         0.92          0.26 0.81     0.71

El modelo SGDClassifier se ha guardado en 'modelo_sgd.pkl'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix


# --- Entrenar un modelo GaussianNB para obtener predicciones completas ---
modelo = GaussianNB()
modelo.fit(X_train, y_train)

# Predecir en todas las muestras del conjunto de prueba
y_pred_full = modelo.predict(X_test)
print("Número de muestras en y_test:", len(y_test))
print("Número de predicciones en y_pred_full:", len(y_pred_full))

# --- Calcular las métricas para cada categoría (one-vs-all) ---
# Convertir y_test a array (si no lo es) y obtener el listado de clases presentes
y_test_array = np.array(y_test)
clases = np.unique(np.concatenate((y_test_array, y_pred_full), axis=0))

# Calcular la matriz de confusión
cm = confusion_matrix(y_test_array, y_pred_full, labels=clases)
print("\nMatriz de confusión:")
print(cm)

# Crear un DataFrame para almacenar las métricas por clase
metricas = pd.DataFrame(index=clases, columns=['Sensibilidad', 'Especificidad', 'F1', 'Accuracy'])

# Para cada clase (one-vs-all), calcular:
for idx, clase in enumerate(clases):
    TP = cm[idx, idx]                  # Verdaderos positivos para la clase
    FN = cm[idx, :].sum() - TP           # Falsos negativos para la clase
    FP = cm[:, idx].sum() - TP           # Falsos positivos para la clase
    TN = cm.sum() - (TP + FP + FN)         # Verdaderos negativos para la clase

    sensibilidad = TP / (TP + FN) if (TP + FN) > 0 else 0
    especificidad = TN / (TN + FP) if (TN + FP) > 0 else 0
    f1 = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
    accuracy = (TP + TN) / cm.sum()

    metricas.loc[clase] = [sensibilidad, especificidad, f1, accuracy]

print("\nMétricas por cada categoría de 'Topic_c':")
print(metricas)

Número de muestras en y_test: 314
Número de predicciones en y_pred_full: 314

Matriz de confusión:
[[ 21   4   2   9]
 [ 15   8   1  10]
 [  1   0  20   7]
 [ 37  17  33 129]]

Métricas por cada categoría de 'Topic_c':
  Sensibilidad Especificidad   F1 Accuracy
0         0.58          0.81 0.38     0.78
1         0.24          0.93 0.25     0.85
2         0.71          0.87 0.48     0.86
3         0.60          0.73 0.70     0.64


In [ ]:
import joblib

modelo.fit(X_train, y_train)

# Guardar el modelo en un archivo (por ejemplo, 'modelo_gnb.pkl')
joblib.dump(modelo, 'modelo_gnb.pkl')
print("Modelo guardado con éxito!")


Modelo guardado con éxito!


In [ ]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from openai import Client

# Configura tu API key 
client = Client(api_key="API")

# Define el prompt según lo proporcionado
prompt = """
Tendrás un rol de clasificador de comentarios de una publicación relacionada con la vacuna contra el VPH.
Sólo debes responder con un valor numérico.
No tienes permitido responder otra cosa que no sean números. Las clasificaciones son:

0: El comentario tiene una postura contraria a la vacuna contra el VPH (antivacuna).
1: El comentario tiene una postura a favor de la vacuna contra el VPH (provacuna).
2: El comentario refleja una duda o dudas relacionadas con la vacuna contra el VPH.
3: El comentario habla de cualquier otra cosa.

Trata de interpretar las intenciones de las personas, ya que se trata de comentarios de Facebook.
Si no puedes clasificar, tu respuesta debe ser "3".

Ahora, clasifica el siguiente comentario, teniendo en cuenta que tu respuesta es solo un número:
"""

def zero_shot_classify(comment: str) -> str:
    """
    Clasifica un comentario usando el prompt definido.
    
    Parámetros:
        comment: Comentario a clasificar.
        
    Retorna:
        La clasificación asignada (un número como string).
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": comment}
    ]
    
    try:
        # Se utiliza el modelo "gpt-4" con temperatura 0 para respuestas deterministas
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
            max_tokens=1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error al clasificar el comentario: {e}")
        return "Error"

# ----- MODELO ZERO-SHOT -----
# Carga el DataFrame con las columnas 'Comment' (comentario a clasificar) y 'Topic_c' (etiqueta verdadera)
df_raw = pd.read_excel('data.xlsx')  # Ajusta la ruta a tus datos

# Selecciona solo los primeros 10 comentarios
df_subset = df_raw

# Aplica el clasificador zero-shot a cada comentario de los primeros 10
df_subset['predicted_topic'] = df_subset['Comment'].apply(zero_shot_classify)

print("Clasificaciones realizadas (Zero-Shot) - Primeros 10:")
print(df_subset[['Comment', 'predicted_topic']])

# ----- EVALUACIÓN -----
# Convertir ambos arreglos a string
y_true = df_subset['Topic_c'].astype(str).values
y_pred = df_subset['predicted_topic'].astype(str).values

# Obtén la lista completa de clases presentes
classes = np.unique(np.concatenate((y_true, y_pred), axis=0))


# Calcula la matriz de confusión
cm = confusion_matrix(y_true, y_pred, labels=classes)
print("\nMatriz de Confusión:")
print(cm)

# Calcula las métricas one-vs-all para cada categoría
metrics = pd.DataFrame(index=classes, columns=['Sensibilidad', 'Especificidad', 'F1', 'Accuracy'])
for idx, cls in enumerate(classes):
    TP = cm[idx, idx]                         # Verdaderos positivos
    FN = cm[idx, :].sum() - TP                  # Falsos negativos
    FP = cm[:, idx].sum() - TP                  # Falsos positivos
    TN = cm.sum() - (TP + FN + FP)              # Verdaderos negativos

    sensibilidad = TP / (TP + FN) if (TP + FN) > 0 else 0
    especificidad = TN / (TN + FP) if (TN + FP) > 0 else 0
    f1 = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
    accuracy = (TP + TN) / cm.sum()

    metrics.loc[cls] = [sensibilidad, especificidad, f1, accuracy]

print("\nMétricas por cada categoría :")
print(metrics)


Clasificaciones realizadas (Zero-Shot) - Primeros 10:
                                                Comment predicted_topic
0     Ya salen los religiosos a decir que solo deben...               0
1                  Prevencion Importante control medico               1
2                               Prevención primaria ! 😍               1
3                Todo lo que es prevención,es excelente               1
4                               Y para los niños cuando               2
...                                                 ...             ...
1561  Deberian vacunar a todas las mujeres, la vacun...               1
1562  Ministerio de Ministerio de Salud del Perú por...               3
1563                         ya vez es desde los 9 años               3
1564  En el. Centro de salud de. Mi ciudad me dijero...               3
1565                                   Si ala vacuna :V               1

[1566 rows x 2 columns]

Matriz de Confusión:
[[128   2  20  13]
 [  3  90  30  6